In [1]:
import joblib
import folium
import branca.colormap as branca_folium_cm
import numpy as np

In [3]:
datas = joblib.load("./scores/Bio_score_1950to2100step10.pickle")
datas.describe()

,x,y,norm
count,192000.000000,192000.000000,192000.000000
mean,45.000000,40.000000,0.500003
std,8.659074,28.867228,0.288676
min,30.250000,-9.750000,0.000005
25%,37.625000,15.125000,0.250004
50%,45.000000,40.000000,0.500003
75%,52.375000,64.875000,0.750001
max,59.750000,89.750000,1.000000


In [4]:
def make_df_map_plotly(_datas):
    colormap = branca_folium_cm.LinearColormap( colors=['green','red'], vmin=0, vmax=1)
    df = _datas[['x','y','norm','geometry']].copy()
    df['geometry']= df['geometry'].buffer(1)
    df['fill_color']= df['norm'].apply(lambda x: colormap(x))
    #datas_geojson = datas_geojson.to_json()
    df.set_geometry('geometry',inplace=True)
    df = df.set_crs("4326")

    df.reset_index(inplace=True)
    df['Year'] = df['time'].apply(lambda x: x.year)
    return df

plot_df = make_df_map_plotly(datas)

In [8]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook_connected'


fig = px.scatter_mapbox(
    plot_df,
    lat= 'x', 
    lon="y",
    color='norm',
    color_continuous_scale = ['blue','green','red'] ,  #'Turbo',  ['green','red']
    size= np.ones(plot_df.shape[0])*50,
    opacity=0.04,
    mapbox_style='carto-positron',  
    animation_frame = 'Year',
    zoom=3,
)

#fig#.show()

In [7]:
# path_name = "C:/Users/demo/Downloads/"
fig.write_html("./maps/animated_plot.html")

-------------------------------------------------------